In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [2]:
# Load ANN trained model, scaler encoder, label encoder
model=load_model('model.h5')

# Load encoder
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# example input data
input_data = {
    'CreditScore':700,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 5000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember': 1,
    'EstimatedSalary': 10000

}

Feature Engineering

In [4]:
# convert into 2D 
geo_data = np.array([[input_data['Geography']]])
geo_data

array([['France']], dtype='<U6')

In [5]:
geo_encoder = one_hot_encoder_geo.transform([[input_data['Geography']]])
geo_encoder

d:\Projects\udemycourse\ANNModel\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [6]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_data_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_data_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
gender_res = label_encoder_gender.transform([input_data['Gender']])

In [9]:
gender_res

array([1])

In [14]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,Male,40,3,5000,2,1,1,10000


In [16]:
input_data_df = pd.concat([input_data_df.reset_index(drop=True), geo_data_df], axis=1)
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,France,Male,40,3,5000,2,1,1,10000,1.0,0.0,0.0


In [19]:
input_data_df = input_data_df.drop('Geography',axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,Male,40,3,5000,2,1,1,10000,1.0,0.0,0.0


In [20]:
input_data_df['Gender'] = gender_res
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,40,3,5000,2,1,1,10000,1.0,0.0,0.0


Predict IP data with model

In [27]:
#Scaling input data
input = scaler.transform(input_data_df)
input

array([[ 0.50178794,  0.91324755,  0.10479359, -0.69539349, -1.13841561,
         0.80843615,  0.64920267,  0.97481699, -1.57229902,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
#Prediction
output = model.predict(input)
output

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step


array([[0.00868477]], dtype=float32)

In [29]:
output[0][0]

np.float32(0.008684771)

In [30]:
if output > 0.5 :
    print('The customer likely stay in bank')
else:
    print('The customer will leave the blank')

The customer will leave the blank


In [31]:
one_hot_encoder_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [32]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)